In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [4]:
# Part 1: Acquiring links to each album's page
url_efy = "https://singpraises.net/collections/music-from-especially-for-youth" # SingPraises page with all EFY albums
r1 = requests.get(url_efy)
soup = BeautifulSoup(r1.text)

In [5]:
# acquiring all the links on the page that link to EFY albums
links = soup.find_all("a", href = re.compile("/collections/en/efy*"), class_ = "text-link")
urls = []
# the href attribute only has the part of the URL after the website's base URL, so we have to add that in
for i in range(0, len(links)):
    urls.append("https://singpraises.net/" + links[i]["href"])
urls

['https://singpraises.net//collections/en/efy-1984-discovering-new-horizons?edition=1984',
 'https://singpraises.net//collections/en/efy-1985-let-your-light-shine?edition=1985',
 'https://singpraises.net//collections/en/efy-1986-lovin-life?edition=1986',
 'https://singpraises.net//collections/en/efy-1987-sailin-home?edition=1987',
 'https://singpraises.net//collections/en/efy-1988-win-the-race?edition=1988',
 'https://singpraises.net//collections/en/efy-1989-forever-my-friend?edition=1989',
 'https://singpraises.net//collections/en/efy-1990-learning-for-myself?edition=1990',
 'https://singpraises.net//collections/en/efy-1991-walk-with-me?edition=1991',
 'https://singpraises.net//collections/en/efy-1992-of-one-heart?edition=1992',
 'https://singpraises.net//collections/en/efy-1993-sharing-the-light?edition=1993',
 'https://singpraises.net//collections/en/efy-1994-serving-with-strength?edition=1994',
 'https://singpraises.net//collections/en/efy-1995-return-with-honor?edition=1995',
 'ht

In [9]:
# Part 2: Acquiring data for each album
tracks = {"Title":[], "Artist1":[], "Words1":[], "Music1":[],
          "Arrangement":[],
          "Album":[], "TrackNum":[], "TotalTracks":[], "YouTube": []} # main 
yt = {"YouTube":[], "Length":[]} # 

In [10]:
for i in range(0, len(urls)):
    r2 = requests.get(urls[i])
    soup2 = BeautifulSoup(r2.text)
    titles = soup2.find_all("div", class_ = "title") # title class contains song titles
    credits = soup2.find_all("div", class_ = "credits-group") # credits-group class contains artist, words, music, and arrangement credits
    title = soup2.find("title").get_text() # title tag contains album's title
    for j in range(0, len(titles)):
        tracks["Title"].append(titles[j].get_text().replace("\n", "").replace("  ", ""))
        try:
            tracks["Artist1"].append(credits[j * 6].find_all("p")[0].get_text().replace("Artist: ", "").strip())
        except Exception:
            print("Missing artist credit on " + titles[j].get_text().replace("\n", "").replace("  ", "") + " in album " + title)
            tracks["Artist1"].append(None)
        try:
            tracks["Words1"].append(credits[j * 6 + 1].find_all("p")[0].get_text().replace("Words:", "").replace("\n", "").strip())
        except Exception:
            print("Missing words credit on " + tracks["Title"][j] + " in album " + title)
            tracks["Words1"].append(None)
        try:
            tracks["Music1"].append(credits[j * 6 + 1].find_all("p")[1].get_text().replace("Music:", "").replace("\n", "").strip())
        except Exception:
            print("Missing music credit on " + tracks["Title"][j] + " in album " + title)
            tracks["Music1"].append(None)
        try:
            tracks["Arrangement"].append(credits[j * 6 + 1].find_all("p")[2].get_text().replace("Arrangement: ", "").strip()) #append(credits[j * 4 + 3].get_text())
        except Exception:
            tracks["Arrangement"].append(None)
        try:
            tracks["YouTube"].append(soup2.find_all("li", class_ = "icon-link video-link")[j].find_all("a")[0]["href"])
            yt["YouTube"].append(soup2.find_all("li", class_ = "icon-link video-link")[j].find_all("a")[0]["href"])
        except Exception:
            tracks["YouTube"].append(None)
        tracks["Album"].append(title)
        tracks["TrackNum"].append(j + 1)
        tracks["TotalTracks"].append(len(titles))

Missing artist credit on Steady and Sure(Instrumental Track) in album EFY 2008: Steady and Sure (2008, 2008b)
Missing artist credit on A Woman’s Heart(Instrumental Track) in album EFY 2008: Steady and Sure (2008, 2008b)
Missing artist credit on One by One(Instrumental Track) in album EFY 2008: Steady and Sure (2008, 2008b)
Missing artist credit on Jesus, the Very Thought of Thee(Instrumental Track) in album EFY 2008: Steady and Sure (2008, 2008b)


In [12]:
tracks = pd.DataFrame(tracks)

,Title,Artist1,Words1,Music1,Arrangement,Album,TrackNum,TotalTracks,YouTube
0,Lovin’ Life,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1986: Lovin’ Life (1986),1,1,None
1,Sailin’ Home,Michael Webb,Michael Webb,Michael Webb,None,EFY 1987: Sailin’ Home (1987),1,10,None
2,Stay Out of Deep Water,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1987: Sailin’ Home (1987),2,10,None
3,Footprints in the Sand,Julie de Azevedo,Barbara Tanner; Randy Thorderson,Barbara Tanner; Randy Thorderson,None,EFY 1987: Sailin’ Home (1987),3,10,None
4,Ups and Downs,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1987: Sailin’ Home (1987),4,10,None
...,...,...,...,...,...,...,...,...,...
395,Faces,Gabrielle McKeon,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),8,12,https://www.youtube.com/watch?v=q7_ush0maMo
396,He’ll Never Let You Go,Patch Crowe,Nik Day,Nik Day,None,EFY 2019: Trust With All Thine Heart (2019),9,12,https://www.youtube.com/watch?v=_LuUgtyyqs8
397,Count Me In,Josh Dalley,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),10,12,https://www.youtube.com/watch?v=l96DDM35HbQ
398,A Missionary’s Prayer,Anna Richey,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),11,12,https://www.youtube.com/watch?v=GT140NtlkSA


In [ ]:
tracks

In [14]:
# Part 3: Getting YouTube length data
for i in range(0, len(yt["YouTube"])):
    r3 = requests.get(yt["YouTube"][i])
    soup3 = BeautifulSoup(r3.text)
    # the next line finds the text code PT#M##S and extracts the numbers; the first # stands for the number of minutes and the second ## stands for seconds
    x, y = soup3.find_all("div", class_ = "watch-main-col")[0].find_all("meta", itemprop = "duration")[0]["content"].replace("PT", "").replace("S", "").split("M")
    yt["Length"].append(int(x) * 60 + int(y))

In [ ]:
yt

In [16]:
tracks_total = pd.merge(pd.DataFrame(tracks), pd.DataFrame(yt), how = "left")

In [17]:
tracks_total

,Title,Artist1,Words1,Music1,Arrangement,Album,TrackNum,TotalTracks,YouTube,Length
0,Lovin’ Life,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1986: Lovin’ Life (1986),1,1,None,NaN
1,Sailin’ Home,Michael Webb,Michael Webb,Michael Webb,None,EFY 1987: Sailin’ Home (1987),1,10,None,NaN
2,Stay Out of Deep Water,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1987: Sailin’ Home (1987),2,10,None,NaN
3,Footprints in the Sand,Julie de Azevedo,Barbara Tanner; Randy Thorderson,Barbara Tanner; Randy Thorderson,None,EFY 1987: Sailin’ Home (1987),3,10,None,NaN
4,Ups and Downs,Kenneth Cope,Kenneth Cope,Kenneth Cope,None,EFY 1987: Sailin’ Home (1987),4,10,None,NaN
...,...,...,...,...,...,...,...,...,...,...
395,Faces,Gabrielle McKeon,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),8,12,https://www.youtube.com/watch?v=q7_ush0maMo,215.0
396,He’ll Never Let You Go,Patch Crowe,Nik Day,Nik Day,None,EFY 2019: Trust With All Thine Heart (2019),9,12,https://www.youtube.com/watch?v=_LuUgtyyqs8,270.0
397,Count Me In,Josh Dalley,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),10,12,https://www.youtube.com/watch?v=l96DDM35HbQ,217.0
398,A Missionary’s Prayer,Anna Richey,Tyler Castleton; Wayne Burton,Tyler Castleton; Wayne Burton,None,EFY 2019: Trust With All Thine Heart (2019),11,12,https://www.youtube.com/watch?v=GT140NtlkSA,235.0


In [19]:
# tracks_total.to_csv("efy_tracks.csv")